In [ ]:
! pip install pypdf langchain_openai langchain_community faiss-cpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
# set openai key

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("openai")

In [ ]:
# import and define LLM

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")
llm.invoke("what can you help me with?")

AIMessage(content="As an AI, I can assist you with a variety of tasks including:\n\n1. Answering questions: I can provide information on a wide range of topics.\n2. Setting reminders: I can remind you about important dates, tasks, or events.\n3. Sending messages: I can help you draft and send messages.\n4. Making lists: I can help you create and manage to-do lists.\n5. Providing news updates: I can give you the latest news on topics that interest you.\n6. Providing weather updates: I can tell you the current weather or forecast in your location or elsewhere.\n7. Recommending music or movies: I can suggest songs, films, TV shows based on your preferences.\n8. Providing information about restaurants, shops, or other places: I can give you details about specific venues.\n9. Telling jokes or fun facts: I can entertain you with some humor or interesting trivia.\n10. Assisting with language translation: I can translate phrases or sentences from one language to another.\n\nAnd much more! Just

In [ ]:
# parse & load PDF

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("/cookies.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [ ]:
# load to knowledge base

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

faiss_index.save_local('cookies')


In [ ]:
 # Loading the saved embeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI

embeddings = OpenAIEmbeddings()

loaded_vectors = FAISS.load_local("cookies", embeddings, allow_dangerous_deserialization=True)

# ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever= loaded_vectors.as_retriever()
)

chat_history = []
def rag(query):

  response = qa({"question":query, "chat_history":chat_history})
  chat_history.append((query, response['answer']))


  return response['answer'].strip()

In [ ]:
rag('How long do I beat the eggs and sugar for?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'In all three recipes, the sugars and egg (or eggs) should be beaten together for about 1-2 minutes until they become pale and light.'

In [ ]:
rag('I want to make all 3 recipes and compare them, make me a grocery list')

'Of course, here\'s a combined grocery list for all three recipes:\n\n- 2 3/4 cups butter, at room temperature or plant-based butter\n- 1 cup coconut oil (optional for vegan cookies)\n- 3 cups packed brown sugar (you can choose between dark or golden brown)\n- 1 cup white sugar\n- 2 large eggs\n- 6 tsp vanilla\n- 5 1/2 cups all-purpose flour\n- 2 tsp baking soda\n- 1 cup boiling water\n- 1 1/2 cups old-fashioned or quick oats\n- 3 cups chopped dark chocolate, vegan chocolate chips, or semi-sweet chocolate chips\n- 1/2 cup chopped walnuts or pecans (optional)\n- 1/4 tsp fine salt for the thin and chewy chocolate chip cookies\n- 1/2 tsp fine salt for the vegan oatmeal chocolate chunk cookies\n- 1/2 tsp fine salt for the chocolate chunk cookies\n- Parchment paper\n- 2 large bowls\n\nNote: The salt is divided into 1/4 tsp and 1/2 tsp, so you will need to buy enough salt for the total 1 1/4 tsp required across all recipes. Also, the amount of chocolate can be adjusted based on your preferen

In [ ]:
rag('How long does the vegan recipe take and at what temperature?')

'The oven should be preheated to 350˚F and the cookies should be baked for 12-15 minutes.'